In [136]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import re
from datetime import datetime
datetime.strptime('1 Jun 2005  1:33PM', '%d %b %Y %I:%M%p')

lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday() 


# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

odata_df = pd.read_csv('C:\\Users\\jduan\\Downloads\\strike.csv')
combine = [odata_df]

odata_df.shape

odata_df = odata_df.drop(['Closed Date Time','Geolocation (Latitude)','Geolocation (Longitude)','Incident: Incident Name'], axis=1)
odata_df.head(2)


# daymap = {0:'Monday', 1:'Tuesday',2:'Wednesday', 3:'Thursday',4:'Friday',5:'Saturday',6: 'Sunday'}
odata_df['Created Weekday']=  odata_df['Created Date Time'].apply(lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday()).astype(int)
# odata_df['Created Weekday'] = odata_df['Created Weekday'].map(daymap)

odata_df['Occurrence Weekday']=  odata_df['Occurrence Date'].dropna().apply(lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday()).astype(int)
# odata_df['Occurrence Weekday'] = odata_df['Occurrence Weekday'].map(daymap)

odata_df['Expiration Weekday']=  odata_df['Expiration Date'].dropna().apply(lambda x:  datetime.strptime(x, '%d.%m.%Y %H:%M').weekday()).astype(int)
# odata_df['Expiration Weekday'] = odata_df['Expiration Weekday'].map(daymap)

odata_df.tail(10)




odata_df.loc[odata_df['City'] == 'Nationwide', 'Area'] = odata_df.loc[odata_df['City'] == 'Nationwide', 'Area'].fillna(2)
odata_df['Area'] = odata_df['Area'].replace(np.nan, 0)

odata_df[['Area','City']].head(10)


odata_df = odata_df.drop(odata_df[(odata_df['Source complete'] == 'Manual')].index)
odata_df['Source complete'] = odata_df['Source complete'].replace(np.nan, 'R360')
odata_df['Source complete'] = odata_df['Source complete'].replace(['NC4 AB','NC4 DHL','NC4 GFP','NC4 NIMC','NIMC' ],'NC4')
odata_df['Source complete'] = odata_df['Source complete'].replace('Anvil','ANVIL')

odata_df['Source complete'].describe()

odata_df[['Source complete', 'Area']].groupby(['Source complete'], as_index=False).mean()
odata_df[['Source complete', 'Severity Number']].groupby(['Source complete'], as_index=False).mean()



#odata_df = pd.concat([odata_df,odata_df['Categories'].str.split(',').apply(pd.Series)],axis=1)
odata_df['Strike Category'] = odata_df['Categories'].apply(lambda x: [a for a in x.split(',') if "strike" in a.lower()][0].strip()) 
odata_df = odata_df.drop(['Categories'], axis=1)


odata_df[['Strike Category', 'Severity Number']].groupby(['Strike Category'], as_index=False).mean()


odata_df.tail(10)

import pycountry

countrymapping = {country.name: country.alpha_2 for country in pycountry.countries}
odata_df['Country ISO'] = odata_df['Country'].map(countrymapping)


countrymap = {"Democratic People's Republic of Korea":'KP','Curacao':'CW','The former Yugoslav Republic of Macedonia':'MK','Taiwan':'TW','United Republic of Tanzania':'TZ','Kosovo':'XK','Bahamas, The':'BS','Republic of Moldova':'MD','State of Palestine':'PS', 'Republic of Korea':'KR','Venezuela (Bolivarian Republic of)':'VE', 'Democratic Republic of the Congo':'CD','Bolivia (Plurinational State of)':'BO','Russia':'RU','Czech Republic':'CZ','Iran (Islamic Republic of)':'IR'}
odata_df['Country ISO'] = odata_df['Country ISO'].fillna(odata_df['Country'].map(countrymap))

odata_df = odata_df.drop(odata_df[(odata_df['Country ISO'].isnull())].index)


odata_df = odata_df.drop(['Country','City'], axis=1)


odata_df['Created Date Time'] = pd.to_datetime(odata_df['Created Date Time']).dt.date
odata_df['Occurrence Date'] = pd.to_datetime(odata_df['Occurrence Date']).dt.date
odata_df['Expiration Date'] = pd.to_datetime(odata_df['Expiration Date']).dt.date




#odata_df['Strike Month'] = odata_df['Current Cleaned Headline'].str.findall(r'\s(Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sep|September|Oct|October|Nov|November|Dec|December[w.-]+)\s')
#odata_df['Strike Weekday'] = odata_df['Current Cleaned Headline'].str.findall(r'Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday')
#daymap = {'Monday':0,'Tuesday':1,'Wednesday':2, 'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}

#odata_df['Strike Weekday'] = odata_df['Strike Weekday'].map(lambda x: ".".join(x)) 

#odata_df['Strike Weekday'] = odata_df['Strike Weekday'].map(daymap)

#combos1=["today","tomorrow","after tomorrow"]

import datetime
now=datetime.datetime.now()
def print_list(my_list):
   for ele in my_list:
       print(ele)
       
def _from_to(my_list):
   ret = []
   for i in range(len(my_list)):
       for j in my_list[:i]:
           ret.append(j + "-" + my_list[i])
   return ret
   
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
brev_month = [i[0:3] for i in months]
#months.extend([i + "." for i in brev_month])
months.extend(brev_month)
months = months + ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
#months = months + [i + "," for i in months] + [i + "." for i in months] +  [i + "-" for i in months]+ [i + ";" for i in months]
numbers= list([str(i) for i in range(1,32)])
numbers= numbers+[i+"st" for i in numbers if i in ('1','21','31')]+[i+"nd" for i in numbers if i in ('2','22')]+[i+"rd" for i in numbers if i in('3','23')]+[i+"th" for i in numbers if i not in ('1','21','31','2','22','3','23')]
#numbers = numbers + [i + "," for i in numbers] + [i + "." for i in numbers] +  [i + "-" for i in numbers]+ [i + ";" for i in numbers]
combos2 = [month + "/" + number for number in numbers for month in months] + [month + " " + number for number in numbers for month in months] + [number + " " + month for month in months for number in numbers] + [number + " of " + month for month in months for number in numbers]
combos2 = combos2 + [i + "," for i in combos2] + [i + "." for i in combos2] +[i + "-" for i in combos2]
del months, numbers
def match_check(strike_text, matching_list):
   text_array = strike_text.split()
   length = len(text_array)
   return [text_array[i] for i in range(length) if text_array[i] in matching_list] + [(text_array[i - 1] + " " + text_array[i]) for i in range(length) if (text_array[i - 1] + " " + text_array[i]) in matching_list]
#odata_df['Strike Date'] = odata_df['Current Cleaned Headline'].map(lambda x:  match_check(str( x)  , combos2)) 

sample_incident1 = "Sunday, March 15-16."
sample_incident2 = "Sunday, 3/15."
sample_incident3 = "Sunday, March 15."
sample_incident4 = "Sunday, Apr 15 and 16."
sample_incident5 = "Sunday, Oct 1st,"
sample_incident6 = "Sunday, 10 of May."

print(match_check(sample_incident1,combos2))
print(match_check(sample_incident2,combos2))
print(match_check(sample_incident3,combos2))
print(match_check(sample_incident4,combos2))
print(match_check(sample_incident5,combos2))
print(match_check(sample_incident6,combos2))



#odata_df['Strike Date'] = odata_df['Current Cleaned Headline'].map(lambda x:  match_check(str( x)  , combos2)) 






#odata_df.tail(10)

#combos3=["Saturday","Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Sat","Sun","Mon","Tue","Wed", "Thu", "Fri"]
#combos3=combos3+[i+"," for i in combos3]+[i+"." for i in combos3]



[]
['3/15.']
['March 15.']
['Apr 15']
['Oct 1st,']
[]


['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '1st', '21st', '31st', '2nd', '22nd', '3rd', '23rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th', '24th', '25th', '26th', '27th', '28th', '29th', '30th']


['January/1', 'February/1', 'March/1', 'April/1', 'May/1', 'June/1', 'July/1', 'August/1', 'September/1', 'October/1', 'November/1', 'December/1', 'Jan/1', 'Feb/1', 'Mar/1', 'Apr/1', 'May/1', 'Jun/1', 'Jul/1', 'Aug/1', 'Sep/1', 'Oct/1', 'Nov/1', 'Dec/1', '1/1', '2/1', '3/1', '4/1', '5/1', '6/1', '7/1', '8/1', '9/1', '10/1', '11/1', '12/1', 'January/2', 'February/2', 'March/2', 'April/2', 'May/2', 'June/2', 'July/2', 'August/2', 'September/2', 'October/2', 'November/2', 'December/2', 'Jan/2', 'Feb/2', 'Mar/2', 'Apr/2', 'May/2', 'Jun/2', 'Jul/2', 'Aug/2', 'Sep/2', 'Oct/2', 'Nov/2', 'Dec/2', '1/2', '2/2', '3/2', '4/2', '5/2', '6/2', '7/2', '8/2', '9/2', '10/2', '11/2', '12/2', 'January/3', 'February/3', 'March/3', 'April/3', 'May/3', 'June/3', 'July/3', 'August/3', 'September/3', 'October/3', 'November/3', 'December/3', 'Jan/3', 'Feb/3', 'Mar/3', 'Apr/3', 'May/3', 'Jun/3', 'Jul/3', 'Aug/3', 'Sep/3', 'Oct/3', 'Nov/3', 'Dec/3', '1/3', '2/3', '3/3', '4/3', '5/3', '6/3', '7/3', '8/3', '9/3', 

[]
[]
[]
['Apr 15']
[]
[]


[]
[]
[]
[]
[]
[]


In [ ]:
from dateutil.parser import parse

def is_date(token):
    try:
        parse(token)
        return True
    except ValueError:
        return False